In [1]:
import pickle

seed = None
with open('_step1_randomseed.pk', 'rb') as f:
    seed = pickle.load(f)

import numpy as np
np.random.set_state(seed)

In [2]:
import pandas as pd
import Utils.dataset_processing as dp

PFT_dataframe = pd.read_csv('Data/_step1_end2end_labeled_datatable.csv')

In [3]:
sup_data_path = 'Data/train_data_labeled.jsonl'

PFT_datatable = PFT_dataframe.values.tolist()
Super_datatable = dp.read_json_to_table(sup_data_path)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-large")

In [5]:
PFT_loader = dp.dataset_loader(PFT_datatable, tokenizer)
V_loader = dp.dataset_loader(Super_datatable, tokenizer)

In [6]:
traindata = dp.load_data(PFT_loader, 16)
vdata = dp.load_data(V_loader, 16)

In [7]:
import robust_loss
import model
import torch
import Utils.train as tr

model = model.RoBERTa_Classify("roberta-large", 3).cuda()
for param in model.roberta.parameters():
    param.requires_grad = True
newloss = robust_loss.NCEandMAE(alpha = 12.0, beta = 0.1, num_classes=3)
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = 1e-5, weight_decay = 1e-5)
model.compile(opt, newloss)

tr.train_classify_model(
    model,
    3,
    traindata,
    vdata,
    save_best = 'loss'
) 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch   1 /   3


  0%|0/352[00:00<?]

val_loss:1.3791, val_acc:0.7348
>>>>>>>>>>>>>>>>>>>>>>>Best result:1.3791>>>>>>>>>>>>>>>>>>>>>>>
Epoch   2 /   3


  0%|0/352[00:00<?]

val_loss:1.0911, val_acc:0.7716
>>>>>>>>>>>>>>>>>>>>>>>Best result:1.0911>>>>>>>>>>>>>>>>>>>>>>>
Epoch   3 /   3


  0%|0/352[00:00<?]

val_loss:1.0762, val_acc:0.7665
>>>>>>>>>>>>>>>>>>>>>>>Best result:1.0762>>>>>>>>>>>>>>>>>>>>>>>


In [8]:
import numpy as np
import pickle

seed = np.random.get_state()

with open("_step2_randomseed.pk", "wb") as file:
    pickle.dump(seed, file)

In [9]:
torch.save(model.state_dict(), "_PFTed.model")